In [1]:
from data_parser import DataParser
from datetime import datetime
import pandas as pd

In [2]:
dp = DataParser(filename='data/GBPUSD.csv')

In [3]:
start_datetime = datetime.strptime('20021001 223600', '%Y%m%d %H%M%S')
end_datetime = datetime.strptime('20120228 183600', '%Y%m%d %H%M%S')

In [4]:
dp.filter_by_date(start_datetime, end_datetime)
dp.sample_by_time('40min', 2160000000000)
Xn,yn = dp.get_close_price_numpy_array(0, 21, 21, 30,36)

In [5]:
Xn1 = dp.tranform_X_to_returns(Xn)

In [6]:
Xn2 = dp.transform_X_with_IWMA(Xn1)

In [7]:
X_norm, y_norm = dp.normalize_data(Xn2, yn)

In [8]:
print(X_norm.shape)
print(y_norm.shape)

(2429, 20)
(2429,)


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_X, test_X, train_y, test_y = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)

In [11]:
from ann import ANN

In [12]:
from GA import GeneticAlgorithm

In [13]:
def fitness(chromosome):
    #chromosomes have 8 bits. first 4 bits are the hidden layer 1, next 4 bits are hidden layer 2. but chromosome is a numpy array
    hidden_layer_1 = 0
    hidden_layer_2 = 0
    for i in range(4):
        hidden_layer_1 += chromosome[i] * (2 ** (3-i))
        hidden_layer_2 += chromosome[i+4] * (2 ** (3-i))

    hidden_layer_1 = max(1, hidden_layer_1)
    hidden_layer_2 = max(1, hidden_layer_2)
    ann = ANN(hidden_layer_1, hidden_layer_2, 20, 1, 'tanh', 'adam', 500, 42)
    ann.fit(train_X, train_y)
    y_pred = ann.predict(test_X)
    nc = ann.num_correct(test_y, y_pred)
    return nc

In [14]:
ga = GeneticAlgorithm(population_size=30, chromosome_length=8, crossover_rate=0.8, mutation_rate=0.2, num_generations=20, stall_generations=15)
ga.fitness_function = fitness
best_chromosome, best_fitness = ga.run()

Generation 1 of 20


Generation 2 of 20
Generation 3 of 20
Generation 4 of 20
Generation 5 of 20
Generation 6 of 20
Generation 7 of 20
Generation 8 of 20
Generation 9 of 20
Generation 10 of 20
Generation 11 of 20
Generation 12 of 20
Generation 13 of 20
Generation 14 of 20
Generation 15 of 20
Generation 16 of 20
Generation 17 of 20
Generation 18 of 20
Generation 19 of 20
Generation 20 of 20
Best Chromosome: [0 1 0 0 1 0 1 0]
Best Fitness: 266


In [18]:
print(best_chromosome)

[0 1 0 0 1 0 1 0]


In [19]:
hidden_layer_1 = 0
hidden_layer_2 = 0
for i in range(4):
    hidden_layer_1 += best_chromosome[i] * (2 ** (3-i))
    hidden_layer_2 += best_chromosome[i+4] * (2 ** (3-i))

hidden_layer_1 = max(1, hidden_layer_1)
hidden_layer_2 = max(1, hidden_layer_2)
ann = ANN(hidden_layer_1, hidden_layer_2, 20, 1, 'tanh', 'adam', 100, 42)
ann.fit(train_X, train_y)
y_pred = ann.predict(test_X)
nc = ann.num_correct(test_y, y_pred)
print(nc)

266


In [20]:
print(ann.num_wrong(test_y, y_pred))

220
